<img src="https://imagedelivery.net/Dr98IMl5gQ9tPkFM5JRcng/3e5f6fbd-9bc6-4aa1-368e-e8bb1d6ca100/Ultra" alt="Image description" width="160" />

<br/>

# Introduction to Contextual AI Platform using the Python Client

Contextual AI lets you create and use generative AI agents. This notebook introduces an end-to-end example workflow for creating a Retrieval-Augmented Generation (RAG) agent for a financial use case. This notebook uses the python client.

This notebook covers the following steps:
- Creating a Datastore
- Ingesting Documents
- Creating an RAG Agent
- Querying an RAG Agent
- Evaluating an RAG Agent
- Improving the RAG Agent (Updating prompt)

The notebook can be run in under 15 minutes. The full documentation is available at [docs.contextual.ai](https://docs.contextual.ai/)

To run this notebook interactively, you can open it in Google Colab. However, make sure to copy over the additional files.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ContextualAI/examples/blob/main/01-getting-started/end-to-end-example.ipynb)

In [ ]:
%pip install contextual-client matplotlib

In [ ]:
import os
import requests
import json
from pathlib import Path
from typing import List, Optional, Dict
from IPython.display import display, JSON
import pandas as pd
from contextual import ContextualAI

Add your API Key here. If you are using our US Cloud, you can leave the default values.
If you are on another instance, modify `base_url` to the appropriate address.

In [ ]:
#Setup API key
#os.environ["CONTEXTUAL_API_KEY"] = API_KEY  # You can store the API key is stored as the environment variable 

client = ContextualAI(
    api_key=os.environ.get("CONTEXTUAL_API_KEY"),
    base_url="http://api.contextual.ai/v1"
)  

## Step 1: Create your Datastore


You will need to first create a datastore for your agent using the  /datastores endpoint. A datastore is secure storage for data. Each agent will have it's own datastore for storing data securely.

In [ ]:
result = client.datastores.create(name="Demo_fin_rag")
datastore_id = result.id
print(f"Datastore ID: {datastore_id}")

## Step 2: Ingest Documents into your Datastore

You can now ingest documents into your Agent's datastore using the /datastores endpoint. Documents must be a PDF or HTML file.


I am using a example PDF. You can also use your own documents here. If you have very long documents (hundreds of pages), processing can take longer.

In [ ]:
if not os.path.exists('data'):
    os.makedirs('data')

if not os.path.exists('data/Apple.pdf'):
    print(f"Fetching data/Apple.pdf")
    response = requests.get("https://raw.githubusercontent.com/ContextualAI/examples/refs/heads/main/01-getting-started/data/Apple.pdf")
    with open('data/Apple.pdf', 'wb') as f:
        f.write(response.content)

Let's ingest the file to start the parsing/extraction process.

In [ ]:
with open('data/Apple.pdf', 'rb') as f:
    ingestion_result = client.datastores.documents.ingest(datastore_id, file=f)
    document_id = ingestion_result.id
    print(f"Successfully uploaded to datastore {datastore_id}")

Once ingested, you can view the list of documents, see their metadata, and also delete documents.

In [ ]:
metadata = client.datastores.documents.metadata(datastore_id = datastore_id, document_id = document_id)
print("Document metadata:", metadata)

You can also export the parsed document for use outside of Contextual.  

**Note:** It may take a few minutes for the document to be ingested and processed. 

In [ ]:
getdocs = client.datastores.documents.get_parse_result(datastore_id=datastore_id, document_id=document_id,output_types="markdown-document")
getdocs.markdown_document

## Step 3: Create your Agent

Next let's create an Agent.

In [ ]:
app_response = client.agents.create(
    name="Demo Fin Agent",
    description="Financial Research Agent using 10k and 10q documents",
    datastore_ids=[datastore_id]
)
agent_id= app_response.id
print(f"Agent ID created: {agent_id}")

## Step 4: Query your Agent

Let's query our agent to see if its working. The required information is the agent_id and messages.  

In [ ]:
query_result = client.agents.query.create(
    agent_id=agent_id,
    messages=[{
        "content": "what was the sales for Apple in 2022",
        "role": "user"
    }]
)
print(query_result.message.content)

There is much more information you can access from the query result. You view the attributions in an image format. 

In [ ]:
import base64
import io
from PIL import Image
import matplotlib.pyplot as plt

def display_base64_image(base64_string):
    # Decode base64 string
    img_data = base64.b64decode(base64_string)
    
    # Create PIL Image object
    img = Image.open(io.BytesIO(img_data))
    
    # Display using matplotlib
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
    return img

# Retrieve the first referenced document `retrieval_contents[0]
ret_result = client.agents.query.retrieval_info(message_id = query_result.message_id, agent_id=agent_id,content_ids=[query_result.retrieval_contents[0].content_id])
print("\nRetrieval Info:", ret_result)
base64_string = ret_result.content_metadatas[0].page_img
img = display_base64_image(base64_string)

## Step 5: Evaluate your Agent



Evaluation endpoints allow you to evaluate your Agent. Contextual support using a variety of evalulation frameworks, such as RAGAS, and different evaluation platforms, such as Langfuse. You will find additional examples around evaluation in this repo.

For those using unit tests, we also offer our `lmunit` endpoint, get more details [here](https://contextual.ai/blog/lmunit/) 

LMUnit is specifically trained for evaluating natural language unit tests and provides:
* Scores on a continuous 1-5 scale
* Consistent evaluation across different criteria
* Better performance than general-purpose LLMs like GPT-4
* Ability to add rubrics to evaluation
* Apply thresholds to get binary scores, e.g., if score is greater than 2.5 is 1 else 0

Let's start with a simple example to understand how LMUnit works:

In [ ]:
response = client.lmunit.create(
                    query="What was Apple's total net sales for 2022?",
                    response="For the quarter ended December 31, 2022, Apple reported total net sales of $117,154 million, representing a decrease from $123,945 million in the same quarter of the previous year (December 25, 2021).[1]() This quarterly data represents only one quarter of Apple's fiscal year. The full-year 2022 total net sales figure is not available in the provided documentation. The sales breakdown for Q4 2022 was: - Products: $96,388 million[1]() - Services: $20,766 million[1]() - Total: $117,154 million[1]() To provide the total net sales for the full fiscal year 2022, we would need additional quarterly data from the other three quarters of the year.",
                    unit_test="Does the response avoid unnecessary information?"
                )
print(response)

## Step 6: Improving your Agent

Contexual AI provides many settings for improving overall performance including on the retrieval side or the generation side. A simple example is modifying the system prompt, you can modify the agent as follows.
For additional ways to improve the agent, look over the [Improve Agent Performance](https://github.com/ContextualAI/examples/tree/main/06-improve-agent-performance) notebook.


### 6.1 Revising the system prompt


After initial testing, you may want to revise the system prompt. Here I have an updated prompt with additional information in the critical guidelines section.  

In [67]:
system_prompt = '''
You are a financial AI assistant created by Contextual AI to help analysts with 10-K forms and related financial documentation. Provide precise, accurate information sourced exclusively from the provided documents.

## Guidelines
Source-Only: Use only information from provided documentation. No opinions, speculation, or external knowledge.
Financial Precision: Use exact terminology, figures, and descriptions as they appear in the documents.
Concise & Relevant: Keep responses focused on financial analysis needs. Prioritize quantitative data and material business information.
Exact Terms: Use financial acronyms and abbreviations exactly as they appear (GAAP, EBITDA, SG&A, etc.).
Markdown: Apply formatting for financial tables, lists, and data presentations.

## Response Protocol
Direct Answer: Answer the question with relevant financial data, then STOP. Avoid unnecessary explanations.
Missing Data: If information isn't available, respond: "I don't have relevant documentation containing that financial information."
Identity/Non-Financial Queries: Respond: "I am a financial AI assistant created by Contextual AI specializing in 10-K analysis. I don't have relevant documentation about that topic, but feel free to ask about financial information from the provided documents!"

Your role is to extract and present financial information accurately to support analyst decision-making, not to provide investment advice.
'''

Let's now update the agent. And verify that changes by checking the agent metadata.

In [ ]:
client.agents.update(agent_id=agent_id, system_prompt=system_prompt)

agent_config = client.agents.metadata(agent_id=agent_id)
print (agent_config.system_prompt)

## Next Steps

In this Notebook, we've created a RAG agent in the finance domain, evaluating the agent, and improved it for better performance. To find further methods for improving your RAG Agent, check out the [Improvement notebook](https://colab.research.google.com/github/ContextualAI/examples/blob/main/06-improve-agent-performance/improvement-overview.ipynb) in this repo.  

You can learn more at [docs.contextual.ai](https://docs.contextual.ai/).   

Finally, reach out to your account team if you have further questions or issues.